In [3]:
# работает: переводит и пишет в файл. но нужно доделать не только футбол

# загружаем необходимые модули и библиотеки
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from deep_translator import GoogleTranslator

#проверяем доступность сайта
games = ['futbol', 'hokkey', 'basketbol']
links = []

for game in games:
    req = Request(
        url='https://metaratings.ru/prognozy/' + game,
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    website_url = urlopen(req).read()
    soup = BeautifulSoup(website_url, 'lxml')
    soup = soup.find('div', class_='TipsList_TipsList__dDOki')
    news_link = soup.find_all('div', class_='TipsList_TipsBoxTitle__sCRne')
    for news in news_link:
        links.append(
            {
                'link': 'https://metaratings.ru' + news.find('a').get('href'),
                'text': news.find('a').get_text()#.lower().replace('«', '').replace('»', '').replace('— ', '')
            })
df1 = pd.DataFrame(links)
df1

,link,text
0,https://metaratings.ru/prognozy/futbol/atalant...,Прогноз на матч Аталанта – Лечче. Бергамаски о...
1,https://metaratings.ru/prognozy/futbol/pszh-li...,Прогноз на матч ПСЖ – Лилль. Парижане не смогу...
2,https://metaratings.ru/prognozy/futbol/elche-e...,Прогноз на матч Эльче – Эспаньол: ставим на об...
3,https://metaratings.ru/prognozy/futbol/rubin-c...,Прогноз на матч Рубин – ЦСКА. Ждём забитые мячи
4,https://metaratings.ru/prognozy/futbol/salerni...,Прогноз на матч Салернитана – Лацио. В первой ...
...,...,...
121,https://metaratings.ru/prognozy/basketbol/nyu-...,Прогноз на матч «Нью-Йорк» – «Юта». Вторая вст...
122,https://metaratings.ru/prognozy/basketbol/atla...,Прогноз на матч «Атланта» – «Сан-Антонио». «Хо...
123,https://metaratings.ru/prognozy/basketbol/vash...,Прогноз на матч «Вашингтон» – «Индиана». «Уиза...
124,https://metaratings.ru/prognozy/basketbol/orla...,Прогноз на матч «Орландо» – «Майами». Ждём упо...


In [4]:
df2 = pd.DataFrame()
key_words = ['барселона', 'кадис']
key_words = [item.lower() for item in key_words]
df1.text = [item.lower() for item in df1.text]
for line in df1.text:
    if all(word in line for word in key_words):
        df0 = df1[df1.text.str.contains(line)]
        df2 = pd.concat([df2, df0], axis= 0 )
df2

,link,text
16,https://metaratings.ru/prognozy/futbol/barselo...,прогноз на матч барселона – кадис: ставим на о...


In [6]:
#проверяем доступность сайта
news = ''
text = ''
for link in df2.link:
    req = Request(
        url=link,
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    website_url = urlopen(req).read()
    soup = BeautifulSoup(website_url, 'lxml')

    # Extract the title of the article
    title = soup.find("div", class_="workarea-title").text.strip() + ". "

    # Extract the text of the article
    article_body = soup.find("article", class_=None)

    # Check the last p tags for the presence of a tags
    for p in article_body.find_all("p")[-3:]:
        if p.find("b"):
            p.extract()
    #
    #     # Concatenate the text content of the remaining p tags
    #     for p in article_body.find_all("p"):
    #         article_text += p.text.strip() + " "
    #         text = title + article_text
    #     text += '\n'
    # # for item in text.split('\n'):
    # #     if len(item.split(' ')) > 10:
    # #         print (item)
    #     print(text)


    for td in article_body.find_all('p', class_=None):
        news += str(td.text) + ' '
    news += '\n'
# print (news)
with open('C:/Users/vitalii/IdeaProjects/settings/vilomov/gpt_rewrite_in.txt', 'w') as file:
    for item in news.split('\n'):
        translated = GoogleTranslator(source='ru', target='en').translate(item)
        file.write(translated + '\n')

# for item in news.split('\n'):
#     if len(item.split(' ')) > 100:
#         print(item)